In [1]:
import numpy as np
from numpy.polynomial import Polynomial

In [2]:
def random_uniform_complex(radius: float, k: int):
    a = np.random.uniform(-radius, radius, k)
    b = np.random.uniform(-radius, radius, k)
    return a + 1j * b


class RootsNotFoundError(Exception):
    def __init__(self):
        default_message = "Algorithm for finding roots has not converged"
        super().__init__(default_message)


def retry_search(times):
    def decorator(func):
        
        def new_func(*args, **kwargs):
            for i in range(times):
                try:
                    return func(*args, **kwargs)
                except RootsNotFoundError:
                    pass
            return func(*args, **kwargs)
        
        return new_func
    return decorator

In [3]:
def Durand_Kerner_method(poly_coeff: np.array, 
                         max_steps=100,
                         atol=1e-6):
    
    poly_coeff = np.array(poly_coeff)
    poly_degree = len(poly_coeff) - 1
    f = Polynomial(poly_coeff[::-1] / poly_coeff[0])
    
    init_radius = 1 + np.abs(poly_coeff).max()
    roots = random_uniform_complex(init_radius, poly_degree)

    for _ in range(max_steps):
        for var_index in range(poly_degree):
            num = f(roots[var_index])
            # product(xi-xj), i != j
            diff = roots[var_index] - np.delete(roots, var_index)
            den = np.prod(diff)
            roots[var_index] -= num / den
            
        if np.allclose(f(roots), 0, atol=atol):
            return roots
        
    raise RootsNotFoundError()

In [4]:
def Aberth_method(poly_coeff: np.array, 
                  max_steps=100,
                  atol=1e-6):
    
    poly_coeff = np.array(poly_coeff)
    poly_degree = len(poly_coeff) - 1
    f = Polynomial(poly_coeff[::-1] / poly_coeff[0])
    
    init_radius = 1 + np.abs(poly_coeff).max()
    roots = random_uniform_complex(init_radius, poly_degree)

    for _ in range(max_steps):
        
        num = f(roots) / f.deriv()(roots)
        s = np.array([
            # sum(1/a)
            np.sum(np.reciprocal(
                # (xi-xj), i != j
                roots[i] - np.delete(roots, i)
            ))
            for i in range(poly_degree)
        ])

        den = 1 - num * s
        roots -= num / den

        if np.allclose(f(roots), 0, atol=atol):
            return roots
    
    raise RootsNotFoundError()

In [5]:
poly_coeff = [3, -5, 2, -6, 4, 2]

In [6]:
roots = Durand_Kerner_method(poly_coeff)

In [7]:
roots.round(6)

array([-0.317572-0.j      ,  1.604692+0.j      , -0.310227-1.100892j,
       -0.310227+1.100892j,  1.      -0.j      ])

In [8]:
roots = Aberth_method(poly_coeff)

In [9]:
roots.round(6)

array([-0.317572-0.j      ,  1.604692+0.j      , -0.310227-1.100892j,
       -0.310227+1.100892j,  1.      -0.j      ])